In [1]:
import os
import sys
sys.path.append('../../')

from utils import add_api_key
add_api_key()

In [43]:
from typing import List

from models.tools import DBSearchTool
from models.llm.agent.custom_streming_callback import CustomStreamingStdOutCallbackHandler
from models.llm.chain import InstantlyAnswerableDiscriminatorChain, PlanningChain

class ChatAgentModule:
    def __init__(self, verbose=False):
        self.tools = [DBSearchTool()]
        self.instantly_answerable_discriminator = InstantlyAnswerableDiscriminatorChain(verbose=verbose)
        self.planning_chain = PlanningChain(tools=self.tools, verbose=verbose, model='gpt-4-1106-preview')
    
    def run(self, query, chat_history: List[List[str]]=[], project_id=-1):
        """
        Args:
            query (str): 사용자 입력
            chat_history (List[List[str]]): [[사용자 입력, 챗봇 출력], ...]
        """
        self.queue = [] # TODO: 나중에 backend에서 주면 삭제
        self.streaming_callback = CustomStreamingStdOutCallbackHandler(queue=self.queue)
        result = self.instantly_answerable_discriminator.run(query=query, chat_history=chat_history)
        print(f'Instantly answerable: {result}')
        if result['instantly answerable'] == 'yes':
            return result['answer']
        else:
            file_name_text = self.tools[0].get_summary_by_project_id(project_id)
            result = self.planning_chain.run(query=query, database_filename=file_name_text, chat_history=chat_history)
            print(f'Planning result: {result}')
            plan_list = list(result[key] for key in result.keys() if 'step' in key)
            return plan_list

In [44]:
chat_agent = ChatAgentModule(verbose=True)

WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.


In [45]:
chat_agent.run(query='전기차 시장의 2020년과 2021년 규모를 비교해줘')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a GPT, a large language model trained by OpenAI, based on the GPT-3.5 architecture.

As my esteemed AI language assistant, you are tasked with determining whether my input can be answered or not. 

If you can answer, then you should providing me the most efficient and accurate solutions to my queries.

By diligently analyzing my inputs, you must swiftly identify my intent and tailor your responses accordingly.

## RESPONSE FORMAT INSTRUCTIONS
----------------------------
When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you can instantly answer the input.
```json
{
    "instantly answerable": "yes",
    "reason": string \ reason for why you can instantly answer the input,
    "answer": string \ Answer for my input. Always respond in a very natural Korean language. Tone and manner should be friendly. 
}
```

**Option #2:**
Use this if you can not instantly answer t

['Use the Database Search tool to find the market size data for EVs in the year 2020.',
 'Use the Database Search tool to find the market size data for EVs in the year 2021.',
 'Compare the relevant figures from both years to identify any growth or decline in the market size.',
 'Analyze and present the comparison data to find trends or significant changes between the two years.']

In [37]:
from models.llm.chain import DBToolQueryGeneratorChain, PlanAnswerChain, PlanAnswerCriticChain, NewPlanningChain

class PlanExecutor:
    def __init__(self, verbose=False):
        self.tools = [DBSearchTool()]
        self.instantly_answerable_discriminator = InstantlyAnswerableDiscriminatorChain(verbose=verbose)
        self.db_tool_query_generator = DBToolQueryGeneratorChain(verbose=verbose)
        self.plan_answer_chain = PlanAnswerChain(verbose=verbose, model='gpt-4-1106-preview')
        self.plan_answer_critic_chain = PlanAnswerCriticChain(verbose=verbose)
        self.new_planning_chain = NewPlanningChain(verbose=verbose)
        self.tool_dict = self.create_tool_dict(self.tools)
    
    def run(self, plan, project_id=-1):
        """
        Args:
            query (str): 사용자 입력
            chat_history (List[List[str]]): [[사용자 입력, 챗봇 출력], ...]
        """
        result = self.instantly_answerable_discriminator.run(query=plan, chat_history=[])
        if result['instantly answerable'] == 'yes':
            return result['answer']
        else:
            result = self.db_tool_query_generator.run(query=plan)
            search_result = self.execute_tool('Database Search', result['query'])
            plan_answer = self.plan_answer_chain.run(query=plan, context=search_result)
            critic_result = self.plan_answer_critic_chain.run(query=plan, answer=plan_answer['final answer'])
            if critic_result['success'] == 'yes':
                return plan_answer['final answer']
            elif (critic_result['success'] == 'no') or (critic_result['success'] == 'partly'):
                answer = critic_result.get('answer', None)
                feedback = critic_result['feedback']
                plan = self.new_planning_chain.run(previous_command=plan, feedback=feedback)
                print(plan)

    def execute_tool(self, tool_name, tool_input):
        if tool_name in self.tool_dict:
            return self.tool_dict[tool_name].run(tool_input)
        
    def create_tool_dict(self, tools):
        return {tool.name: tool for tool in tools}

In [38]:
plan_executor = PlanExecutor(verbose=True)

WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to mod

In [46]:
plan_executor.run(plan='Use the Database Search tool to find the market size data for EVs in the year 2020.')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a GPT, a large language model trained by OpenAI, based on the GPT-3.5 architecture.

As my esteemed AI language assistant, you are tasked with determining whether my input can be answered or not. 

If you can answer, then you should providing me the most efficient and accurate solutions to my queries.

By diligently analyzing my inputs, you must swiftly identify my intent and tailor your responses accordingly.

## RESPONSE FORMAT INSTRUCTIONS
----------------------------
When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you can instantly answer the input.
```json
{
    "instantly answerable": "yes",
    "reason": string \ reason for why you can instantly answer the input,
    "answer": string \ Answer for my input. Always respond in a very natural Korean language. Tone and manner should be friendly. 
}
```

**Option #2:**
Use this if you can not instantly answer t

In [21]:
from modules.file_to_db.file_processor import FileProcessor

WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.


In [22]:
file_processor = FileProcessor()
file_processor('https://www.hani.co.kr/arti/economy/car/1116579.html')

한겨레 광고 본문 경제 자동차 ‘전기차 시장’ 성장 더뎌졌다…하이브리드는 되레 선전 전기차 성장 둔화, 경기침체·보조금 감축 등 영향 기자 최우리 2023-11-16 14:47 2023-11-16 14:47 SNE리서치 제공 올해 세계 80개국에 등록된 전기차와 하이브리드차의 규모가 약 1377만대가 될 것이라는 전망이 나왔다. 상반기에 예측했던 대수보다 100만대 이상 감소했다. 성장률도 36.4%에서 30.6%로 하향조정됐다. 업계에서는 성장률 둔화가 내년까지 지속할 것으로 본다. 시장조사기관 에스엔이(SNE)리서치는 16일 이같은 전망치를 내놓으면서 내년에도 성장률이 20% 전후로 전보다 더딘 성장을 보일 것이라고 밝혔다. 전기차 시장 성장 둔화는 경기침체로 인해 소비자들이 비싼 전기차 가격에 부담이 커졌고, 보조금 감축과 충전 인프라 부족 문제 등이 맞물리면서 소비 심리가 위축되었기 때문이다. 또 코로나19 기간 자동차 공급난으로 인한 대기 수요가 몰린 효과가 올해 들어 약해지면서 성장률을 둔화시켰다고 에스엔이리서치는 분석했다. 반면 하이브리드차(HEV)의 비중은 기존 전망 보다 더 커질 것으로 보인다. 당초 전기차(49%)-하이브리드차(32%)-플러그인하이브리드차(19%)로 판매 비중을 예측했지만, 하이브리드차는 36%로 높아지는 대신 전기차는 45%로 떨어질 것으로 전망했다. 에스엔이리서치는 “보조금 감축과 폐지로 인해 중국에서 순수 전기차 비중이 감소하고 유럽 지역에서 하이브리드차 비중이 증가한 것으로 보인다. 전 지역에 걸쳐 기존 예상보다 하이브리드차가 감소하지 않은 것으로 분석됐다”고 짚었다. 광고 순수 전기차, 하이브리드차, 플러그인하이브리드 비중 비교. SNE리서치 제공 ※ 이미지를 누르면 크게 볼 수 있습니다. 최우리 기자 ecowoori@hani.co.kr 전기차 전기차 하이브리드 최우리기자 다른 기사 어떠세요 기아·한국GM 최대 실적…완성차 5사 799만대 판매 달성 현대차, 광명 전기차 공장서 신년회…정의선 “지속가능한 미래 열어야”